In [1]:
import datetime

import pandas as pd
import numpy as np
import re
import os
import glob

from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

In [2]:
# Start time ---
day_time = print(datetime.datetime.now().date())
start_time = print(datetime.datetime.now().time())

2019-09-04
00:09:00.938899


In [4]:
# Create function to remove the html code etc. from content 
def clean_up(text):
    text = re.sub('<.*?>', '', text)
    text = re.sub('\\n', '', text)
    text = text.strip()
    return text

In [5]:
# Get data
titles = []
pub_dates = []
types = []
summaries = []
content = []

# Get reference to pages on website
pages = [str(i) for i in range(0,76)]

# Set year
year = 2017
year = str(year)

#For every page
for page in pages:
        
    # Make a get request
    response = get('https://www.fca.org.uk/news/search-results?start=' + page + '1&year=f.Published%20Year%7Cyear%3D' + year + '&sort_by=dmetaZ')

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request:{}; Status code: {}'.format(requests, response.status_code))

    # Parse with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Select all containers from page
    article_containers = soup.find_all('li', class_='search-item')

    # Loop through all containers
    i = 0
    for i in range(0, len(article_containers)):

        # Titles
        title = article_containers[i].a.text
        if title is None:
            titles.append('NaN')
        else: 
            titles.append(clean_up(title))
            
        # Pub date
        pub_date = article_containers[i].find('span', class_='meta-item published-date').text
        if pub_date is None:
            pub_dates.append('NaN')
        else:
            pub_dates.append(clean_up(pub_date))
                    
        # Type
        type_ = article_containers[0].find('span', class_='meta-item type').text
        if type_ is None: 
            types.append('NaN')
        else:
            types.append(clean_up(type_))

        # Summary
        summary = article_containers[i].find('div', class_='search-item__body').text
        if summary is None:
            summaries.append('NaN')
        else: 
            summaries.append(clean_up(summary))
        
        # Content
        link = article_containers[i].find('a')['href']
        
        # Make a get request
        response_link = get(link)
        
        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request:{}; Status code: {}'.format(requests, response_link.status_code))
        
        # Parse with BeautifulSoup
        soup = BeautifulSoup(response_link.text, 'html.parser')
        
        # Select all containers from page
        link_container = str(soup.find_all('div', class_='field-item'))
        if link_container is None:
            content.append('NaN')
        else: 
            content.append(clean_up(link_container))        

In [6]:
# Create dataframe summary of information
df = pd.DataFrame({'title': titles, 
                   'pub_date': pub_dates,
                   'type': types,
                   'summary': summaries,
                   'content': content
                  }
                 )

# Write dataframe to CSV
df.to_csv(r'C:\Users\Harriet Goers\Desktop\UNSW\FCA_data_' + year + '.csv', index=None, header=True)

In [7]:
# End time --- 
endTime = print(datetime.datetime.now().time())

00:33:57.559044
